Antes de empezar, el semestre pasado dí análisis y diseño de algoritmos, en donde vimos cómo hacer tests de primalidad de manera eficiente (aleatorizados), así que decidí implementar ese algoritmo visto en class http://marceloarenas.cl/iic2283-18/clases/alg_teoria_numeros-2.pdf

In [8]:
from math import floor, ceil, log2
from random import randint

# para calcular el mcd entre dos números, visto en clases
# https://github.com/UC-IIC3253/2022/blob/main/presentaciones%20de%20clases/11_alg_teoria_numeros_RSA_parte_1.pdf
def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a % b)

# diapositiva 88 del pdf de IIC2283
# algoritmo que verifica si existe algún número en el rango {i, ..., j}
# tal que n = m^k
def has_integer_root(n, k, i, j):
    # caso base, sólo un candidato
    if i == j:
        if pow(i, k) == n:
            return True
        else:
            return False

    # el rango no tiene elementos
    if i > j:
        return False
    
    # este if es innecesario en teoría, pero es para ordenar
    if i < j:
        p = floor((i + j) // 2) # encontramos la mitad del rango
        val = pow(p, k) # vemos su p^k
        
        if val == n:
            # encontramos un valor que cumple
            return True
        
        if val < n:
            # nos falta más, así que vamos a la mitad mayor
            return has_integer_root(n, k, p + 1, j)

        # vamos a la mitad menor
        return has_integer_root(n, k, i, p - 1)

# diapositiva 87
def is_power(n):
    if n <= 3:
        return False
    for k in range(2, floor(log2(n))): # no es necesario revisar más del log de n (de lo contrario te pasas)
        if has_integer_root(n, k, 1, n):
            return True
    return False

# diapositiva 92, la validez del algoritmo está demostrada allí
# la probabilidad de equivocarse está acotada por 1/2^k
def prime(n):
    k = 20 # hacemos que la probabilidad de equivocarse esté acotada por 1/2^20
    
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    if is_power(n):
        return False

    a = [randint(1, n - 1) for _ in range(k)]
    b = [0 for _ in range(k)]
    for i in range(k):
        if gcd(a[i], n) > 1: # encontramos factores, no puede ser primo
            return False
        b[i] = pow(a[i], (n - 1) // 2, n)
    neg = 0
    for i in range(k):
        if (b[i] % n) == (-1 % n):
            neg += 1
        elif (b[i] % n) != 1:
            return False
    if neg == 0:
        return False
    else:
        return True

# algoritmo para encontrar un número primo alteatorio en cierto rango
# visto en clases que la cantidad de intentos es razonable
def randprime(lower_bound, upper_bound):
    while True:
        number = randint(lower_bound, upper_bound)
        if prime(number):
            return number
    

Habiendonos sacado eso de encima, ahora sí veamos todos algoritmos que ver con sólo este ramo.

In [11]:
# obtiene el tamaño de un bloque de acuerdo a las especificaciones de la tarea
def _get_block_length(n):
    bits = n.bit_length()
    m = (bits - 1) // 64
    n = m * 8
    
    return n

# obtiene el tamaño de un bloque encriptado
# notar que es distinto al tamaño original de cada bloque dado que
# necesitamos que todo número mod n pueda caber en el bloque
# por ende, necesitamos suficiente para la cantidad de bits de n-1
def _get_encrypted_block_length(n):
    bits = (n-1).bit_length()
    return math.ceil(bits/8)
    

In [17]:
import math
import random

# obtiene un primo relativo en mod n
def _find_relative_prime(n):
    while True:
        candidate = random.randint(1, n-1)
        if math.gcd(candidate, n) == 1:
            return candidate

class RSAReceiver:
    def __init__(self, bit_len):
        self.bit_len = bit_len
        self._generate_keys() # generamos las llaves

    def get_public_key(self):
        return self.pk

    def decrypt(self, ciphertext):
        array = ciphertext
        n = _get_encrypted_block_length(self.secret_key[1]) # obtenemos el tamaño de bloques
        
        
        decoded_array = bytearray()
        for i in range(0, len(array), n):
            block = array[i: i+n]
            decoded_array += self._decode_block(block) # decodificamos cada bloque
        return decoded_array.decode('UTF-8')
    
    def _generate_keys(self):
        # necesitamos por lo menos que sean de tamaño (en bits) de bit_len
        p = randprime(2 ** self.bit_len, 2 ** (self.bit_len + 1))
        while True:
            q = randprime(2 ** self.bit_len, 2 ** (self.bit_len + 1))
            if q != p: # si son iguales no funciona exactamente igual el pequeño teorema de Fermat!
                break
        n = p * q 
        phi = (p - 1)*(q - 1)
        
        d = _find_relative_prime(phi)
        e = pow(d, -1, phi)
        
        self.public_key = (e, n)
        self.secret_key = (d, n)
    
        # esta es la llave codificada de acuerdo al formato PEM
        self.pk = self._encode_number(e) + self._encode_number(n)

    # decodificamos el bloque de acuerdo a la llave secreta
    def _decode_block(self, block):
        coded_number = int.from_bytes(block, 'big')
        decoded_number = pow(coded_number, self.secret_key[0], self.secret_key[1])
        desencrypted_block = decoded_number.to_bytes(len(block), 'big')
        return desencrypted_block

    # sirve para serializar un número en el formato PEM
    def _encode_number(self, n):
        bytes_in_n = math.ceil(n.bit_length() / 8)
        encoded_bytes_in_n = bytes_in_n.to_bytes(4, 'big')
        encoded_n = n.to_bytes(bytes_in_n, 'big')
        
        return encoded_bytes_in_n + encoded_n


In [13]:
class RSASender:
    def __init__(self, pk):
        self.pk = pk
        self._decode_public_key() # obtenemos un formato más cómodo para la llave
    
    def encrypt(self, message):  
        n = _get_block_length(self.public_key[1])
        array = bytearray(message, 'UTF-8')

        encoded_array = bytearray()
        for i in range(0, len(array), n): # vamos bloque por bloque
            right_limit = min(len(array), i + n) # no importa si el último bloque es menor
            block = array[i: right_limit]

            encrypted_block = self._encrypt_block(block) # encriptamos cada bloque
            encoded_array += encrypted_block
        
        return encoded_array
    
    # encripta un bloque a partir de la llave pública
    def _encrypt_block(self, block):
        number = int.from_bytes(block, 'big')
        encoded_number = pow(number, self.public_key[0], self.public_key[1])
        encoded_block_len = _get_encrypted_block_length(self.public_key[1])
        encoded_block = encoded_number.to_bytes(encoded_block_len, 'big')
        return encoded_block
    
    # a partir de una llave en PEM, la transforma a un par (e, n)
    def _decode_public_key(self):
        bytes_in_e = int.from_bytes(self.pk[:4], 'big')
        e = int.from_bytes(self.pk[4: 4+ bytes_in_e], 'big')
        bytes_in_n = int.from_bytes(self.pk[4+ bytes_in_e:8+ bytes_in_e], 'big')
        n = int.from_bytes(self.pk[8+ bytes_in_e: 8+ bytes_in_e + bytes_in_n], 'big')
        self.public_key = (e, n)
        

In [18]:
# TEST
#receiver = RSAReceiver(100)
#pk = receiver.get_public_key()
#sender = RSASender(pk)

#text = '1aasawedf' * 10

#encrypted = sender.encrypt(text)
#decrypted = receiver.decrypt(encrypted)
#print(decrypted)